In [ ]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB

In [ ]:
import os
import openai
import numpy as np
import pandas as pd
import json
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding

In [ ]:
openai.api_key = ""

In [ ]:
os.environ["OPENAI_API_KEY"] = ''

In [ ]:
System_Prompt = """
You are an assistant that specializes in summarizing news articles. When the user provides an article or main points, analyze the content carefully to identify and extract the essential facts, key events, main participants, and core themes. Your summary should focus on delivering:

Conciseness: Aim for a brief overview that captures the main points in 2-4 sentences, unless the user specifies a different length.
Objectivity: Refrain from adding personal opinions, interpretations, or any information not in the article.
Clarity: Use simple, direct language for easy readability, focusing on clear, neutral descriptions.
Relevance: Prioritize primary details such as the who, what, when, where, why, and how of the article.
For instance:

If the article discusses a breaking news event, emphasize the incident, the time and location, key people involved, and any immediate outcomes.
For business or financial news, highlight relevant companies, industry developments, market impacts, or financial figures.
For political or policy-related topics, summarize the policy or issue, the stakeholders, and potential impacts on the public.
If the article is complex, use bullet points for clarity if appropriate. Ensure that every summary remains informative and easy to understand, so the user quickly grasps the essential information without needing to read the full article.
"""

In [ ]:
user_message = """
Title: Bagyong 'Kristine,' may posibilidad umano na mahila ng panibagong bagyo para bumalik sa PAR
Body: Inaasahan ng state weather bureau na PAGASA na lalabas na sa Philippine Area of Responsibility (PAR) sa Biyernes ng hapon ang bagyong "Kristine" na nanalasa sa Bicol Region at Luzon. Gayunman, posible umano itong bumalik sa PAR kapag nahila ng paparating na bagong bagyo.

Nitong Huwebes, patungo na si Kristine sa West Philippine Sea makaraang magpaulan sa Bicol Region at malaking bahagi ng Luzon nitong nakaraang mga araw, at kumitil ng nasa 20 katao.

Gayunman, habang papalabas ng PAR si Kristine, isang low pressure area (LPA) ang nabubuo sa silangan ng northeastern Mindanao na inaasahang magiging bagyo pagpasok sa PAR at tatawaging "Leon."

"There is a developing forecast situation wherein Kristine will be looping over the West Philippine Sea on Sunday (October 27) and Monday (October 28) and move eastward or east northwestward towards the general direction of the PAR region,'' ayon sa PAGASA.

Nakasalalay umano sa galaw ng bagong LPA kung ganap itong magiging bagyo at kung mahahatak nito pabalik sa PAR si Kristine, dahil sa tinatawag ng mga meteorologists na Fujiwhara effect.

Ang "interaksyon" ng dalawang bagyo ay depende umano sa magiging lakas at distansya ng mga ito sa isa't isa.

Ayon sa Hong Kong Observatory, dapat may layo na 1,350 kilometers sa isa't isa ang dalawang bagyo para mangyari ang Fujiwhara effect.

''The stronger cyclone tends to have a dominant effect on the track of the weaker one. The interaction will end when: there is a stronger influence of a large scale weather system from outside, one of the tropical cyclones weakens or the two cyclones merge,'' ayon sa  Hong Kong Observatory.— FRJ, GMA Integrated News


"""

In [ ]:
struct = [{"role" : "system", "content" : System_Prompt}]
struct.append({"role" : "user", "content": user_message})
chat = openai.ChatCompletion.create(model="gpt-4o-mini", messages = struct)
response = chat.choices[0].message.content
struct.append({"role" : "assistant", "content": response})
print("Assistant:", response)


Assistant: Bagyong "Kristine" ay inaasahang lalabas mula sa Philippine Area of Responsibility (PAR) sa Biyernes ng hapon, matapos magdulot ng malakas na pag-ulan sa Bicol Region at Luzon na nagresulta sa pagkamatay ng 20 katao. Gayunpaman, may posibilidad itong bumalik sa PAR dahil sa paparating na low pressure area (LPA) sa silangan ng northeastern Mindanao, na inaasahang magiging bagyo at tatawaging "Leon." Ang galaw ng mga bagyo ay nakasalalay sa tinatawag na Fujiwhara effect, kung saan ang interaksyon nila ay depende sa kanilang lakas at distansya mula sa isa't isa.
